In [1]:
import numpy as np
from numpy.random import Generator, PCG64
import matplotlib.pyplot as plt
import math
import time
from heateq import Exact, Simulateur

In [2]:
seed = 213731490053398181466621250222036675538
rng = Generator(PCG64(seed))

In [3]:
# Permet d'obtenir un n-échantillon du vecteur aléatoire d'entrée X
def n_echantillon_X(n):
    return np.vstack(
        (
            rng.uniform(-math.pi, math.pi, (3, n)),
            rng.uniform(0.001, 0.009, (1, n)), 
            rng.uniform(-1., 1., (3, n))
        )
    )

In [4]:
# Simulateurs haute et basse fidélité
f = Simulateur(21, 100)
g = Simulateur(3, 20)

In [5]:
# Espérance exacte de la solution continue
mu_exact = Exact().mu
print(mu_exact)

# Espérance exacte de Y = f(X) (solution discrétisée, haute fidélité)
mu_Y_exact = f.mu
print(mu_Y_exact)

# Espérance exacte de Z = g(X) (solution discrétisée, basse fidélité)
mu_Z_exact = g.mu
print(mu_Z_exact)

41.98447216482205
41.916465294707976
33.03617206344313


# Espérance $\mu_Z$ connue

On considère $\mu_Z$ connue. Elle est donnée par la variable `mu_Z_exact`.

## Paramètre optimal $\alpha^*$ indépendant (précalculé)
Estimer $\alpha^*$ de manière indépendante (avec, par exemple, un échantilon de taille `p = 1000`), puis $\rho^2$, et en déduire $\beta=(1+w)(1-\rho^2)$. L'utiliser pour contruire un estimateur par variable de contrôle pour l'espérance de $Y$. Faire `nr = 1000` répétitions pour des budgets $\tilde{n} \in \{5; 10; 20; 50; 100; 200; 500; 1000\}$.

Estimer la réduction de variance à budget équivalent entre l'estimateur par variable de contrôle et l'estimateur Monte Carlo classique (haute fidélité). Est-elle conforme à la valeur de $\beta$ calculée précédemment ?

Tracer l'espérance et l'écart-type des estimateurs par variable de contrôle et Monte Carlo (haute fidélité) à budget équivalent en fonction de $\tilde{n}$. Sur un autre graphe, tracer l'évolution de l'écart-type des estimateurs en fonction de $\tilde{n}$.

In [6]:
nr = 1000
budgets = [5, 10, 20, 50, 100, 200, 1000]

## Paramètre optimal $\alpha^*$ estimé à la volée
Renouveler l'expérience avec $\alpha^*$ estimé avec les échantillons disponibles pour l'estimation par variable de contrôle.

Tracer les mêmes graphes que précédemment. L'estimation de $\alpha^*$ à la volée induit-elle un biais et/ou une dégradation de la réduction de variance ?

## Métamodèle

Mêmes questions en utilisant un métamodèle comme simulateur basse fidélité.
Voir, par exemple, https://openturns.github.io/openturns/latest/auto_meta_modeling/kriging_metamodel/plot_kriging.html

In [9]:
import openturns as ot

In [10]:
# pour faciliter l'évaluation de la même manière que nos simulateurs
class ScalarModel_from_ot:
    def __init__(self, ot_model):
        self.ot_model = ot_model

    def __call__(self, xi):
        return np.array(self.ot_model(xi.T))[:,0]

In [11]:
n_doe = 1000
X_doe = n_echantillon_X(n_doe)
Y_doe = f(X_doe)
dimension = X_doe.shape[0]

In [12]:
basis = ot.ConstantBasisFactory(dimension).build()
covarianceModel = ot.MaternModel([1.0] * dimension, 1.5)
algo = ot.KrigingAlgorithm(X_doe.T, Y_doe[:,None], covarianceModel, basis) # conversion to OpenTurns data structure
algo.run()
res = algo.getResult()
g_mm = ScalarModel_from_ot( res.getMetaModel() )

WRN - Warning! For coherency we set scale upper bounds = [12.544,12.5147,12.5559,0.0159818,3.9946,3.99079,3.99764]
WRN - Starting point is not inside bounds x=[1,1,1,1,1,1,1] bounds=class=Interval name=Unnamed dimension=7 lower bound=class=Point name=Unnamed dimension=7 values=[0.01,0.01,0.01,0.01,0.01,0.01,0.01] upper bound=class=Point name=Unnamed dimension=7 values=[12.544,12.5147,12.5559,0.0159818,3.9946,3.99079,3.99764] finite lower bound=[1,1,1,1,1,1,1] finite upper bound=[1,1,1,1,1,1,1]


## Plusieurs variables de contrôle

Combiner plusieurs simulateurs basse fidélité (supposer $\mu_Z$ connue), et estimer $R^2$.